# QR Code Obfuscation via Modulo 2 Matrix Multiplication

In this document we descibe a method for obfuscating QR codes by considering said codes as Matrices over $\mathbb{Z}/2\mathbb{Z}$, and then multiplying (on the left) by a random sequence of elementary matrices. 

In modulo $2$ arithmetic, these elementary matrices are their own inverses, and thus inverting the sequence of products by elementary matrices is equivalent computing the same product in reverse order. This relation is used to generate a random pair of matrices, whose product is the original QR code. Furthermore, neither the QR code nor the random product can be obtained from either of the generated matrices.

## Sketch of the Method

Given a QR code of $n \times n$ pixels, construct a matrix $A=(a_{ij})$ such that 

$$
    a_{ij} = \left\lbrace \begin{array}{l l}
    1, & \textrm{if the $ij$-th pixel is white} \\
    0, & \textrm{otherwise.}
    \end{array} \right.
$$

The simplest way imaginable to obfuscate $A$ would be by simply generating a random matrix $B$ and then computing two matrices, namely $BA$ and $B^{-1}$, from which $A$ can be recovered by taking $B^{-1}(BA)$. The way we generate this random matrix in the present work is by generating $k$ random elementary matrices, and taking their product. In particular, if $E_k$ is some elementary matrix, then we take $B = E_k \cdot E_{k-1} \cdots E_2 \cdot E_1$.

The problem with this approach is that, while $B$ is easy to compute, keeping track of all the different inverses of the $E_i$, in order to compute $B^{-1}$, may become a computationally expensive task as $k$ grows larger. Additionally, these matrices are not simple to represent visually (as with QR codes). In the following paragraph we present an improvement of this naive approach, which solves both the representation problem and the efficiency problem.

### Elementary Matrices in Modulo 2 Arithmetic

Since the whole purpose of this computation is to encode a matrix of ones and zeros by splitting it into two matrices (which may be represented by ones and zeros as well to resemble a QR code), it's completely reasonable to leave the field of real numbers, and switch over to the commutative ring of $\mathbb{Z}/2\mathbb{Z}$. Now, elementary matrices also becomes matrices of zeros and ones. But additionally, they gain an additional property that allows us to speed up the process of computing $B^{-1}$. Namely, they become _their own inverses_. 

Row switching matrices are already their own inverses (and intuitively this is fairly clear) over $\mathbb{R}$. Row multiplication matrices trivially become their own inverses, because the factor is always $1$. Finally, to see that row addition matrices also become their own inverses, we may compute $E_a ^{kl} \cdot E_a ^{kl}$, where $E_a ^{kl}$ is a row addition matrix that performs the sum $R_k = R_k + R_l$, $k\neq l$.

Let $E^{kl}_a =\big((a^{\, kl})_{ij}\big)$. Then, it's not difficult to see that

$$
(a^{\, kl})_{ij} = \delta _{ij} + \delta _{ik} \delta _{jl}.
$$

Thus, and using the summation convention,

$$
(a^{\, kl})_{in}(a^{\, kl})_{nj} = \big( \delta _{in} + \delta _{ik} \delta _{nl} \big) \left(  \delta _{nj} + \delta _{nk} \delta _{jl} \right),
$$

where sum over $n$ is implicit. Expanding this product gives

$$
(a^{\, kl})_{in}(a^{\, kl})_{nj} = \delta _{in} \delta _{nj} + \delta _{in} \delta _{nk} \delta _{jl} + \delta _{ik} \delta _{nl} \delta _{nj} + \delta _{ik} \delta _{nl} \delta _{nk} \delta _{jl}
$$

Since $k \neq l$, then $\delta _{nl} \delta _{nk} = 0$ in the last term. Furthermore, $ \delta _{in} \delta _{nk} \delta _{jl} = \delta _{ik} \delta _{nl} \delta _{nj}$. Therefore,

$$
(a^{\, kl})_{in}(a^{\, kl})_{nj} = \delta _{in} \delta _{nj} + 2\delta _{in} \delta _{nk} \delta _{jl} = \delta _{in} \delta_{nj} \; \textrm{(mod 2)}
$$

Finally, $E_a ^{kl} \cdot E_a ^{kl} = I \; \textrm{(mod 2)}$.

## Matrix Obfuscation

Employing the fact that these elementary matrices are their own inverses in Modulo $2$ arithmetic, If we generate a random sequence of $q$ elementary matrices and compute $E_q \cdot E_{q-1} \cdots E_2 \cdot E_1 \cdot A$, then in order to retrieve $A$ we must simply consider the matrix given by $E_1 \cdot E_2 \cdots E_{q-1} \cdot E_q$. From now on, we'll call these matrices $C$ and $K$ respectively, such that $K \cdot C = A$. We'll define some of the auxiliary functions below before we perform the computation. For illustrative purposes, we'll be dealing with $21 \times 21$ matrices to represent QR codes.

In [1]:
import numpy as np

def delta(i,j): # Definition of delta function
    if i == j:
        a = 1
    elif i != j:
        a = 0
    return a

def elem_swap(m,n): # Definition of row swap elementary matrix
    A = []
    for i in range(0,21):
        B = 21*[0]
        for j in range(0,21):
            v = delta(i,j)*(1-delta(i,m))*(1-delta(i,n)) + delta(i,m)*delta(j,n) + delta(i,n)*delta(j,m)
            B[j] = v
        A.append(B)
    A = np.array(A,dtype="int32")
    return A

def iden(): # Definition of identity matrix
    A = []
    for i in range(0,21):
        B = 21*[0]
        for j in range(0,21):
            v = delta(i,j)
            B[j] = v
        A.append(B)
    A = np.array(A,dtype="int32")
    return A

def elem_add(m,n): # Definition of row addition matrix
    A = []
    for i in range(0,21):
        B = 21*[0]
        for j in range(0,21):
            v = delta(i,j) + delta(i,m)*delta(j,n)
            B[j] = v
        A.append(B)
    A = np.array(A,dtype="int32")
    return A

Now, we define some functions to choose two random numbers between $0$ and $21$ (not inclusive) and print the arrays.

In [2]:
def index_rchoice(): # Function to choose two random numbers between 0 and 21 (not inclusive)
    a = np.random.choice(range(0,21),2,replace=False)
    return a

def qrprint(arr): # Function to print the arrays using ASCII characters
    line = ""
    for i in range(0,21):
        for j in range(0,21):
            if arr[i][j] == 0:
                line = line + '\u2588\u2588'
            elif arr[i][j] == 1:
                line = line + '\u2591\u2591'
        print(line)
        line = ""

qr = np.array([
[0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0],
[0,1,1,1,1,1,0,1,0,0,0,1,0,1,0,1,1,1,1,1,0],
[0,1,0,0,0,1,0,1,1,1,0,0,0,1,0,1,0,0,0,1,0],
[0,1,0,0,0,1,0,1,0,0,1,1,0,1,0,1,0,0,0,1,0],
[0,1,0,0,0,1,0,1,1,0,1,1,0,1,0,1,0,0,0,1,0],
[0,1,1,1,1,1,0,1,0,1,1,0,1,1,0,1,1,1,1,1,0],
[0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0],
[1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1],
[0,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1],
[0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,1,1,1,1,1,0],
[0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,1],
[0,0,1,1,1,1,1,0,1,0,0,0,0,0,0,1,1,0,0,1,0],
[1,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0,1,1,1,0,1],
[1,1,1,1,1,1,1,1,0,0,1,1,0,1,1,0,1,0,1,1,1],
[0,0,0,0,0,0,0,1,0,1,1,0,1,0,1,1,0,0,1,0,1],
[0,1,1,1,1,1,0,1,1,0,1,1,1,1,1,0,0,0,0,0,1],
[0,1,0,0,0,1,0,1,0,0,1,0,1,0,1,1,0,0,1,0,1],
[0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,1,0,1,1],
[0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,1,1,1,1,1],
[0,1,1,1,1,1,0,1,0,1,1,0,0,0,0,1,1,0,0,1,1],
[0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,1,1,1,0,1],
], dtype="int32")

The array above is the QR code in this example, representing black squares with $0$'s and white squares with $1$'s. By using `qrprint` we can actually _see_ the barcode.

In [3]:
qrprint(qr)

██████████████░░░░░░██████░░██████████████
██░░░░░░░░░░██░░██████░░██░░██░░░░░░░░░░██
██░░██████░░██░░░░░░██████░░██░░██████░░██
██░░██████░░██░░████░░░░██░░██░░██████░░██
██░░██████░░██░░░░██░░░░██░░██░░██████░░██
██░░░░░░░░░░██░░██░░░░██░░░░██░░░░░░░░░░██
██████████████░░██░░██░░██░░██████████████
░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░░░░░░░
██████████░░██████░░░░██░░██░░██░░██░░██░░
████████░░██░░░░██████████████░░░░░░░░░░██
████████████████░░░░██░░██░░████░░██████░░
████░░░░░░░░░░██░░████████████░░░░████░░██
░░████████░░████░░████░░██░░░░██░░░░░░██░░
░░░░░░░░░░░░░░░░████░░░░██░░░░██░░██░░░░░░
██████████████░░██░░░░██░░██░░░░████░░██░░
██░░░░░░░░░░██░░░░██░░░░░░░░░░██████████░░
██░░██████░░██░░████░░██░░██░░░░████░░██░░
██░░██████░░██░░██░░████████████░░░░██░░░░
██░░██████░░██░░██████░░██░░████░░░░░░░░░░
██░░░░░░░░░░██░░██░░░░████████░░░░████░░░░
██████████████░░██████░░██░░░░░░░░░░░░██░░


Now, we can get into the meat of the matter. We define an array `ops`. This array is to hold, in sequence, the matrices that we use to obfuscate the barcode. A random elementary matrix is generated (random in the sense that it may be a row _swap_ matrix or a row _sum_ matrix), the barcode gets multiplied on the left by said matrix, and then the matrix is appended to the `ops` array. This is repeated a thousand times. Finally, we take the representation of the matrix in modulo $2$.

In [4]:
ops = []

for l in range(0,1000): # Main operation loop
    m,n = index_rchoice()
    if np.random.sample() < 1:
        eadd = elem_add(m,n)
        qr = np.dot(eadd,qr)
        ops.append(eadd)
    else:
        eswap = elem_swap(m,n)
        qr = np.dot(eswap,qr)
        ops.append(eswap)

qr = qr % 2 # Representation in modulo 2 of obfuscated barcode

qrprint(qr)

░░░░████░░██░░░░░░██████░░██░░░░░░████░░░░
████████░░░░░░░░░░██████████░░░░██████░░░░
░░░░░░░░░░██░░░░░░██░░██████░░██████░░██░░
░░██████░░░░░░████░░░░░░░░░░████░░████░░░░
██░░░░░░██████░░██░░██████░░██░░██████░░░░
░░░░████░░██░░██░░░░░░██░░██████░░██░░████
░░██░░░░██░░░░██░░░░░░████░░██░░░░░░██████
░░░░████░░░░████░░░░░░████████░░░░████░░░░
░░██░░░░██░░░░██░░░░░░░░██░░██░░░░░░██████
░░░░██████████░░████░░████░░██░░░░██████░░
████░░░░██░░██████░░██████░░████░░░░██████
░░░░░░░░░░░░████░░░░██░░██████░░██░░░░░░██
░░██░░░░██░░████░░██░░░░░░░░░░████░░██░░██
░░██████░░████████████░░░░██░░░░░░░░████░░
████████░░██░░██░░░░██░░██░░██████████████
░░░░░░░░████░░░░░░████░░░░░░░░░░░░░░░░████
████████░░████░░░░████████░░██░░░░██░░░░░░
██░░░░░░██████░░░░░░░░░░██░░░░░░████████░░
██░░██████████████░░██░░████░░░░██░░████░░
░░████████░░████░░████████░░████░░░░░░░░░░
░░██░░░░████░░░░░░░░██░░██░░██░░░░░░░░░░░░


The above matrix is the obfuscated barcode! ($C$ matrix) <br> Now, as per the calculations we've shown above, all we have to do now is reverse the order of the `ops` array, so as to compute the product of all the elementary matrices in reverse order, to obtain the "Key" matrix:

In [5]:
key = ops.reverse() # Operations in reverse order
keymat = iden() # Initializing an identity matrix

for m in range(len(ops)): # Applying multiplications in reverse order to identity matrix
    keymat = np.dot(ops[m],keymat)
keymat = keymat % 2 # Representation in modulo 2 of the key matrix

qrprint(keymat)

██░░██░░██████░░░░░░░░░░██████░░██████████
░░░░░░░░██░░░░░░░░██░░██░░░░██░░██░░░░████
████░░░░░░░░░░████░░██░░██░░░░░░██░░██████
██████░░██░░██░░██░░░░██████████░░░░░░████
████░░░░████░░░░██░░██████░░░░██░░████████
████░░░░████████░░░░░░░░░░██░░░░░░██░░██░░
░░████████░░░░██░░████░░████████░░░░██░░██
██░░░░██░░████░░░░░░██░░██░░██████░░░░░░██
██░░██░░██░░░░░░░░██░░░░░░██░░████░░██░░██
████████░░░░████░░░░░░████░░░░████░░██████
░░░░████░░░░██████░░██░░░░██░░░░░░░░░░░░░░
░░██░░░░░░████░░░░░░██░░████░░██░░░░░░░░██
░░██░░░░██░░░░████░░██░░████░░░░██░░░░██░░
░░░░████░░██░░████░░██░░████░░░░░░░░░░████
░░░░████░░████░░██░░░░████░░░░░░░░░░██░░██
░░░░████░░░░░░██░░██░░░░░░░░░░░░░░░░░░░░░░
██████████████░░██░░████░░░░░░░░░░████░░██
████░░██░░██████░░██████░░░░░░██░░░░██░░░░
░░████████░░░░░░██████████░░░░░░░░░░██░░░░
██░░░░██░░██░░██████░░██░░██░░░░██████░░░░
████░░██░░██████░░████░░██░░██░░░░██████░░


This is the Key matrix! ($K$ matrix). Our final step is to compute $K \cdot C$,

In [6]:
verf = np.dot(keymat,qr)
verf = verf % 2

qrprint(verf)

██████████████░░░░░░██████░░██████████████
██░░░░░░░░░░██░░██████░░██░░██░░░░░░░░░░██
██░░██████░░██░░░░░░██████░░██░░██████░░██
██░░██████░░██░░████░░░░██░░██░░██████░░██
██░░██████░░██░░░░██░░░░██░░██░░██████░░██
██░░░░░░░░░░██░░██░░░░██░░░░██░░░░░░░░░░██
██████████████░░██░░██░░██░░██████████████
░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░░░░░░░
██████████░░██████░░░░██░░██░░██░░██░░██░░
████████░░██░░░░██████████████░░░░░░░░░░██
████████████████░░░░██░░██░░████░░██████░░
████░░░░░░░░░░██░░████████████░░░░████░░██
░░████████░░████░░████░░██░░░░██░░░░░░██░░
░░░░░░░░░░░░░░░░████░░░░██░░░░██░░██░░░░░░
██████████████░░██░░░░██░░██░░░░████░░██░░
██░░░░░░░░░░██░░░░██░░░░░░░░░░██████████░░
██░░██████░░██░░████░░██░░██░░░░████░░██░░
██░░██████░░██░░██░░████████████░░░░██░░░░
██░░██████░░██░░██████░░██░░████░░░░░░░░░░
██░░░░░░░░░░██░░██░░░░████████░░░░████░░░░
██████████████░░██████░░██░░░░░░░░░░░░██░░


We obtain the desired result, using a $21 \times 21$ QR code as an example.

# Final Comments
The method described above can be used to obfuscate **any** array that can be encoded into $1$'s and $0$'s, not just QR codes, which are a simple example of such arrays. Clearly, the implementation is slow as it can get, but it serves the purpose of being illustrative and a proof of concept. To begin with an optimization, instead of dealing with arrays and dot products in `ops`, we could forget them altogether and keep track of the transformations instead, and apply them in purely algebraic fashion, fetching an array in the end only to show results. Additionally, instead of generating each of the random pairs and numbers at runtime for selecting elementary matrices, an arbitrarily large sequence of random pairs may be chosen at initialization, and then used to compute the products, instead of waiting for every new cycle to choose the numbers. In this way, the sequence of random pairs and transformations becomes in itself the "key" for de-obfuscating the code.